## **Applio**
A simple, high-quality voice conversion tool focused on ease of use and performance.

[Support](https://discord.gg/urxFjYmYYh) — [GitHub](https://github.com/IAHispano/Applio) — [Terms of Use](https://github.com/IAHispano/Applio/blob/main/TERMS_OF_USE.md)

<br>

---

<br>

#### **Acknowledgments**

To all external collaborators for their special help in the following areas:
* Hina (Encryption method)
* Poopmaster (Extra section)
* Shirou (UV installer)
* Bruno5430 (AutoBackup code and general notebook maintenance)

#### **Disclaimer**
By using Applio, you agree to comply with ethical and legal standards, respect intellectual property and privacy rights, avoid harmful or prohibited uses, and accept full responsibility for any outcomes, while Applio disclaims liability and reserves the right to amend these terms.

### **Install Applio**
If the runtime restarts, re-run the installation steps.

In [ ]:
# @title Mount Drive
from google.colab import drive
from google.colab._message import MessageError

try:
  drive.mount("/content/drive")
except MessageError:
  print("❌ Failed to mount drive")

In [ ]:
# @title Setup runtime environment
from IPython.display import clear_output
import codecs

encoded_url = "uggcf://tvguho.pbz/VNUvfcnab/Nccyvb/"
decoded_url = codecs.decode(encoded_url, "rot_13")

repo_name_encoded = "Nccyvb"
repo_name = codecs.decode(repo_name_encoded, "rot_13")

LOGS_PATH = f"/content/{repo_name}/logs"
BACKUPS_PATH = f"/content/drive/MyDrive/{repo_name}Backup"

%cd /content
!git config --global advice.detachedHead false
!git clone {decoded_url} --branch 3.4.0 --single-branch
%cd {repo_name}
clear_output()

# Install older python
!apt install python3.11 python3.11-distutils
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 2
!update-alternatives --set python3 /usr/bin/python3.11

print("Installing requirements...")
!curl -LsSf https://astral.sh/uv/install.sh | sh
!uv pip install -q -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu128 --index-strategy unsafe-best-match
!uv pip install -q ngrok jupyter-ui-poll
!npm install -g -q localtunnel &> /dev/null

!python core.py "prerequisites" --models "True" --pretraineds_hifigan "True"
print("✅ Finished installing requirements!")


### **Start Applio**

In [ ]:
# @title Sync with Google Drive
# @markdown 💾 Run this cell to automatically Save/Load models from your mounted drive
# @title
# @markdown This will merge and link your `ApplioBackup` folder from gdrive to this notebook
from IPython.display import display, clear_output
from pathlib import Path

non_bak_folders = ["mute", "reference", "zips", "mute_spin"]
non_bak_path = "/tmp/rvc_logs"


def press_button(button):
  button.disabled = True


def get_date(path: Path):
  from datetime import datetime
  return datetime.fromtimestamp(int(path.stat().st_mtime))


def get_size(path: Path):
  !du -shx --apparent-size "{path}" > /tmp/size.txt
  return open("/tmp/size.txt").readlines().pop(0).split("	")[0] + "B"


def sync_folders(folder: Path, backup: Path):
  from ipywidgets import widgets
  from jupyter_ui_poll import ui_events
  from time import sleep

  local = widgets.VBox([
      widgets.Label(f"Local: {LOGS_PATH.removeprefix('/content/')}/{folder.name}/"),
      widgets.Label(f"Size: {get_size(folder)}"),
      widgets.Label(f"Last modified: {get_date(folder)}")
  ])
  remote = widgets.VBox([
      widgets.Label(f"Remote: {BACKUPS_PATH.removeprefix('/content/')}/{backup.name}/"),
      widgets.Label(f"Size: {get_size(backup)}"),
      widgets.Label(f"Last modified: {get_date(backup)}")
  ])
  separator = widgets.VBox([
      widgets.Label("|||"),
      widgets.Label("|||"),
      widgets.Label("|||")
  ])
  radio = widgets.RadioButtons(
      options=[
          "Save local model to drive",
          "Keep remote model"
      ]
  )
  button = widgets.Button(
      description="Sync",
      icon="upload",
      tooltip="Sync model"
  )
  button.on_click(press_button)

  clear_output()
  print(f"Your local model '{folder.name}' is in conflict with it's copy in Google Drive.")
  print("Please select which one you want to keep:")
  display(widgets.Box([local, separator, remote]))
  display(radio)
  display(button)

  with ui_events() as poll:
    while not button.disabled:
      poll(10)
      sleep(0.1)

  match radio.value:
    case "Save local model to drive":
      !rm -r "{backup}"
      !mv "{folder}" "{backup}"
    case "Keep remote model":
      !rm -r "{folder}"


if Path("/content/drive").is_mount():
  !mkdir -p "{BACKUPS_PATH}"
  !mkdir -p "{non_bak_path}"

  if not Path(LOGS_PATH).is_symlink():
    for folder in non_bak_folders:
      folder = Path(f"{LOGS_PATH}/{folder}")
      backup = Path(f"{BACKUPS_PATH}/{folder.name}")

      !mkdir -p "{folder}"
      !mv "{folder}" "{non_bak_path}" &> /dev/null
      !rm -rf "{folder}"
      folder = Path(f"{non_bak_path}/{folder.name}")
      if backup.exists() and backup.resolve() != folder.resolve():
        !rm -r "{backup}"
      !ln -s "{folder}" "{backup}" &> /dev/null

    for model in Path(LOGS_PATH).iterdir():
      if model.is_dir() and not model.is_symlink():
        backup = Path(f"{BACKUPS_PATH}/{model.name}")

        if model.name == ".ipynb_checkpoints":
          continue

        if backup.exists() and backup.is_dir():
          sync_folders(model, backup)
        else:
          !rm "{backup}"
          !mv "{model}" "{backup}"

    !rm -r "{LOGS_PATH}"
    !ln -s "{BACKUPS_PATH}" "{LOGS_PATH}"

    clear_output()
    print("✅ Models are synced!")

  else:
    !rm "{LOGS_PATH}"
    !ln -s "{BACKUPS_PATH}" "{LOGS_PATH}"
    clear_output()
    print("✅ Models already synced!")

else:
  print("❌ Drive is not mounted, skipping model syncing")
  print("To sync your models, first mount your Google Drive and re-run this cell")

In [ ]:
# @title **Start server**
# @markdown  ### Choose a sharing method:
from IPython.display import clear_output

method = "gradio"  # @param ["gradio", "localtunnel", "ngrok"]
ngrok_token = "If you selected the 'ngrok' method, obtain your auth token here: https://dashboard.ngrok.com/get-started/your-authtoken" # @param {type:"string"}
tensorboard = True #@param {type: "boolean"}

%cd /content/{repo_name}
clear_output()

if tensorboard:
  %load_ext tensorboard
  %tensorboard --logdir logs --bind_all

match method:
  case 'gradio':
    !python app.py --listen --share
  case 'localtunnel':
    !echo Password IP: $(curl --silent https://ipv4.icanhazip.com)
    !lt --port 6969 & python app.py --listen & echo
  case 'ngrok':
    import ngrok
    ngrok.kill()
    listener = await ngrok.forward(6969, authtoken=ngrok_token)
    print(f"Ngrok URL: {listener.url()}")
    !python app.py --listen